In [1]:
import pandas as pd
import requests
import json
from google.cloud import bigquery
from google.oauth2 import service_account
from pandas.io import gbq

In [2]:
#url da api
url = "https://api.clickup.com/api/v2/team/31090318/time_entries"

In [3]:
#parametros para a requisição da api
query = {
  "start_date": "0",
  "end_date": "0",
  "assignee": "0",
  "include_task_tags": "true",
  "include_location_names": "true",
  "space_id": "0",
  "folder_id": "0",
  "list_id": "0",
  "task_id": "0",
  "custom_task_ids": "true",
  "team_id": "123"
}

headers = {
  "Content-Type": "application/json",
  "Authorization": "pk_48937455_H7RMXQBM3BKKW34P9P71OUBQTFX25Q2P"
}

response = requests.get(url, headers=headers)

data = response.json()

In [4]:
#quebrar as linhas dos dicionários em colunas
def extract(dic,key=None):
    adjust = {}
    for k,v in dic.items():
        if isinstance(v,dict):
            v = extract(v,key=k if not key else f"{key}_{k}" )
            adjust = {**adjust,**v}
        else:
            kk = k if not key else f"{key}_{k}"
            adjust[kk]=v
    return adjust

def extract_sub_dicts(dic): #[{},{}]
    adjust = []
    for r in dic:
        if isinstance(r,dict):
            adjust.append(extract(r))
    return adjust

In [8]:
#mostrar a data e extrair para um csv
data["data"]
dt = extract_sub_dicts(data["data"])
df = pd.DataFrame.from_dict(dt)
df
df.to_csv('data_api.csv',index=False)

In [9]:
#enviar data para um csv 
my_data_api = pd.read_csv('data_api.csv', encoding='latin-1')
print(my_data_api)

                     id  task_id                          task_name  \
0   3222015379098859951  37quyh6             Write a knowledge base   
1   3222015981904230852  37quyhw           Hire brilliant engineers   
2   3222016125986961865  37quyh1             Brainstorming meetings   
3   3222016328806726097  37quyhf        Acquire funding for scaling   
4   3222018368412875290  37quyhf        Acquire funding for scaling   
5   3222018762979441196  37quyh4                             Mockup   
6   3223742009521489770  37quyhy                     Plan the build   
7   3223742076831680365  37quyh2                                MVP   
8   3223742184256194417  37quyhq  Organize teams and delegate tasks   
9   3223742755117746040  37quyhj                   Product meetings   
10  3223742959900444541  37quyhj                   Product meetings   
11  3223743226272302979  37quyh9                    Scale marketing   
12  3223744009516631951  37quyhm               Showcase the product   
13  32

In [10]:
#enviar para o google cloud big query
my_data_api.to_gbq(destination_table='api_clickup.datateste',project_id='project-api-clickup',if_exists='append')

100%|██████████| 1/1 [00:00<?, ?it/s]
